In [115]:
import pandas as pd
import numpy as np

In [116]:
precipitation_df=pd.read_csv(r"data\Cuba_Precipitation_with_Nodes (1) (2).csv")
adj_matrix=pd.read_csv(r"data\Adjacency_Matrix______0_80.csv", index_col=0)

In [117]:
precipitation_df["time"]=pd.to_datetime(precipitation_df["time"])

In [118]:
precipitation_df.head()

,node,lat,lon,time,precipitation
0,1,22.75,-83.75,1979-01-01,0.000000
1,1,22.75,-83.75,1979-01-02,16.168200
2,1,22.75,-83.75,1979-01-03,4.264466
3,1,22.75,-83.75,1979-01-04,0.000000
4,1,22.75,-83.75,1979-01-05,0.000000


In [119]:
precipitation_df=precipitation_df.sort_values(by=["node", "time"])
precipitation_df.reset_index(drop=True,inplace=True)

In [120]:
precipitation_df

,node,lat,lon,time,precipitation
0,1,22.75,-83.75,1979-01-01,0.000000
1,1,22.75,-83.75,1979-01-02,16.168200
2,1,22.75,-83.75,1979-01-03,4.264466
3,1,22.75,-83.75,1979-01-04,0.000000
4,1,22.75,-83.75,1979-01-05,0.000000
...,...,...,...,...,...
661595,40,20.25,-74.25,2024-04-09,0.000000
661596,40,20.25,-74.25,2024-04-10,0.000000
661597,40,20.25,-74.25,2024-04-11,0.000000
661598,40,20.25,-74.25,2024-04-12,0.000000


In [121]:
print(precipitation_df[["node","lat","lon"]])


        node    lat    lon
0          1  22.75 -83.75
1          1  22.75 -83.75
2          1  22.75 -83.75
3          1  22.75 -83.75
4          1  22.75 -83.75
...      ...    ...    ...
661595    40  20.25 -74.25
661596    40  20.25 -74.25
661597    40  20.25 -74.25
661598    40  20.25 -74.25
661599    40  20.25 -74.25

[661600 rows x 3 columns]


In [122]:
nodes=precipitation_df["node"].values
missing_nodes=set(nodes)-set(adj_matrix.index)

In [123]:
missing_nodes
#Seems there is no missing links 

set()

In [124]:
precipitation_df["precipitation_diff"]=precipitation_df.groupby("node")["precipitation"].diff()
precip_diff_df=precipitation_df.dropna(subset=["precipitation_diff"])
# precip_values

In [125]:
precip_diff_df.head(20)

,node,lat,lon,time,precipitation,precipitation_diff
1,1,22.75,-83.75,1979-01-02,16.168200,16.168200
2,1,22.75,-83.75,1979-01-03,4.264466,-11.903734
3,1,22.75,-83.75,1979-01-04,0.000000,-4.264466
4,1,22.75,-83.75,1979-01-05,0.000000,0.000000
5,1,22.75,-83.75,1979-01-06,0.000000,0.000000
6,1,22.75,-83.75,1979-01-07,0.000000,0.000000
7,1,22.75,-83.75,1979-01-08,0.000000,0.000000
8,1,22.75,-83.75,1979-01-09,0.020123,0.020123
9,1,22.75,-83.75,1979-01-10,9.862288,9.842165
10,1,22.75,-83.75,1979-01-11,0.000000,-9.862288


In [126]:
adj_matrix.head()

,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
node,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### STEP 2

In [168]:
pivot_df = precipitation_df.pivot(index='time', columns='node', values='precipitation')

full_range = pd.date_range(start='1979-01', end='2023-12', freq='M')
pivot_df = pivot_df.reindex(full_range)
pivot_df=pivot_df.fillna(0)

C:\Users\gacha\AppData\Local\Temp\ipykernel_11244\2697736483.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  full_range = pd.date_range(start='1979-01', end='2023-12', freq='M')


In [169]:
# pivot_df=pivot_df.copy()
# pivot_df["month"]=pivot_df.index.month

# pivot_df['month_sin'] = np.sin(2 * np.pi * pivot_df['month'] / 12)
# pivot_df['month_cos'] = np.cos(2 * np.pi * pivot_df['month'] / 12)
# pivot_df["year"]=pivot_df.index.year

# pivot_df["year_norm"]=(pivot_df["year"]-pivot_df["year"].min())/(pivot_df["year"].max()-pivot_df["year"].min())

In [170]:
node_id=1
neighbors=adj_matrix.loc[node_id]
linked_nodes=neighbors[neighbors==1].index.tolist()
print(f"Node {node_id} neighbors: {linked_nodes}")

Node 1 neighbors: ['2', '11']


In [171]:
def get_node_series(node_id,precipitation_df):
    return precipitation_df[precipitation_df["node"]==node_id][["time","precipitation_diff"]].reset_index(drop=True)

In [172]:
mode_series=get_node_series(node_id,precipitation_df)

In [173]:
linked_nodes = [int(nb) for nb in linked_nodes]

In [174]:
all_series = [pivot_df[node_id].values]

In [175]:

for nb in linked_nodes:
    all_series.append(pivot_df[nb].values)



In [176]:
X=np.stack(all_series,axis=1)
print(X.shape)

(539, 3)


In [177]:
# X = pivot_df.drop(columns=['month','year']).values
# print(X.shape)

In [178]:
def create_sequences(data,time_index,window_size=90):
    X,y,times=[],[],[]
    for i in range(len(data)-window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size,0])
        times.append(time_index[i+window_size])
    return np.array(X),np.array(y),np.array(times)

x_seq,y_seq,t_seq=create_sequences(X,pivot_df.index)
print(x_seq.shape,y_seq.shape)

(449, 90, 3) (449,)


In [201]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization,Dropout

In [180]:
num_features=x_seq.shape[2]

In [235]:
model=Sequential([
    LSTM(64,input_shape=(30,num_features),return_sequences=False),
    Dense(32,activation="relu",kernel_initializer="he_normal"),
    Dense(1)
])
model.compile(optimizer="adam",loss='mse')
model.summary()

Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_24 (LSTM)                  │ (None, 64)             │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_58 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,521 (76.25 KB)

 Trainable params: 19,521 (76.25 KB)

 Non-trainable params: 0 (0.00 B)

In [236]:
train_mask = (t_seq >= pd.to_datetime("1979-01-01")) & (t_seq <= pd.to_datetime("2014-12-31"))
test_mask  = (t_seq >= pd.to_datetime("2015-01-01")) & (t_seq <= pd.to_datetime("2023-12-31"))

X_train, y_train = x_seq[train_mask], y_seq[train_mask]
X_test, y_test   = x_seq[test_mask],  y_seq[test_mask]




In [237]:
history=model.fit(X_train,y_train,
                  validation_data=(X_test,y_test),
                  epochs=50,
                  batch_size=64,
                  verbose=1
                  )

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 131ms/step - loss: 111.4657 - val_loss: 45.9990
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 106.4723 - val_loss: 43.9239
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 101.3915 - val_loss: 42.4136
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 97.9496 - val_loss: 45.7752
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 96.3967 - val_loss: 46.4955
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 96.3676 - val_loss: 46.6891
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 95.6160 - val_loss: 45.5553
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 95.1976 - val_loss: 44.5346
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 94.9794 - val_loss: 43.9082
Epoch 10/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 94.7122 - val_loss: 44.1570
Epoch 11/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 94.4106 - val_loss: 44.3262
Epoch 12/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 9

In [238]:
loss = model.evaluate(X_test, y_test)
print("Test MSE:", loss)

y_pred = model.predict(X_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 44.2903
Test MSE: 44.290279388427734
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


In [240]:
aligned_series = pivot_df[node_id].reindex(t_seq).values

last_value = aligned_series[train_mask][-1]
y_pred_inverse=[]
for diff in y_pred.flatten():
    next_value=last_value+diff
    y_pred_inverse.append(next_value)
    last_value=next_value

y_pred_inverse=np.array(y_pred_inverse)


In [241]:
y_test_inverse=[]
last_value=aligned_series[train_mask][-1]
for diff in y_test.flatten():
    nect_value=last_value+diff
    y_test_inverse.append(next_value)
    last_value=next_value
y_test_inverse=np.array(y_test_inverse)

In [242]:
results_df=pd.DataFrame({
    "date":t_seq[test_mask],
    "node":node_id,
    "actual":y_test_inverse,
    "forecast":y_pred_inverse
})
results_df.to_csv("forecasts_node_{node_id}.csv",index=False)
print("Saved results for Node", node_id)

Saved results for Node 1
